In [1]:
from sentence_transformers import SentenceTransformer
import torch
import os
## 读取json文件
from tqdm import tqdm
import pandas as pd
import numpy as np
from FlagEmbedding import FlagReranker
import subprocess
import time


## 读取文件
model_embedding = SentenceTransformer('../Model/yangjhchs/acge_text_embedding', device='cuda')
model_reranker = FlagReranker('../Model/AI-ModelScope/bge-reranker-v2-m3', use_fp16=True)

# 转换为一维的NumPy数组
def retriever_top(similarity,num_top,text_list):
    retrieve_dict = {}
    arr = np.array(similarity).flatten()
    # 找到排序后的索引
    sorted_indices = arr.argsort()
    # 因为argsort()返回的是从小到大排序的索引，所以我们需要从末尾开始取
    top3_indices = sorted_indices[-num_top:][::-1]  # 反转切片以获得最大的三个索引
    # 获取对应的值
    top3_values = arr[top3_indices]
    # 输出结果
    n = 0
    for i, v in zip(top3_indices, top3_values):
        retrieve_dict[n] = {}
        retrieve_dict[n]['Index'] = i
        retrieve_dict[n]['score'] = v
        retrieve_dict[n]['content'] = text_list[i]
        n = n+1
    return retrieve_dict

def retriever_rerank(reranker,retrieve_dict,question_intense,num_top_rerank):
    rerank_list = []
    for i in range(len(retrieve_dict)):
        rerank_list.append([str(question_intense), retrieve_dict[i]['content']])
    scores = reranker.compute_score(rerank_list, normalize=True)
    #print(scores) # [0.00027803096387751553, 0.9948403768236574]

    rerank_dict = {}
    arr = np.array(scores).flatten()
    # 找到排序后的索引
    sorted_indices = arr.argsort()
    # 因为argsort()返回的是从小到大排序的索引，所以我们需要从末尾开始取
    top3_indices = sorted_indices[-num_top_rerank:][::-1]  # 反转切片以获得最大的三个索引
    # 获取对应的值
    top3_values = arr[top3_indices]
    # 输出结果
    n = 0
    for i, v in zip(top3_indices, top3_values):
        rerank_dict[n] = {}
        rerank_dict[n]['Index'] = i
        rerank_dict[n]['score'] = v
        rerank_dict[n]['content'] = retrieve_dict[i]['content']
        n = n+1
    return rerank_dict

def retriever(model,reranker, question, Index, text_list, num_top = 10,num_top_rerank = 3):
    question_embeddings = model.encode(question, normalize_embeddings=True)
    similarity = Index @ question_embeddings.T
    retrieve_dict = retriever_top(similarity,num_top,text_list)
    rerank_dict = retriever_rerank(reranker,retrieve_dict,question,num_top_rerank)
    
    return rerank_dict

def load_message(input_content,requirements,Hierarchy,rag_content):
    if Hierarchy == "需求声明":
        messages = f"<相关文件>:{input_content}</相关文件>\n\n根据相关文件给出的信息,相关的体系文件里是否有提出与‘{requirements}’对应的公司内部的要求？不需要考虑编号的一致性!请给出结论,符合性分析,和带文件名称的相关原文.输出格式为:<结论>:结论\n<符合性分析>:符合性分析\n<原文>:原文"
    elif Hierarchy == "制定流程":
        messages = f"<相关文件>:{input_content}</相关文件>\n\n在上述相关文件中,请就‘是否直接展示了关于{requirements}的完整的管理流程或作业指导书或具体的操作方式?’,进行符合性分析.请给出结论,符合性分析,和带文件名称的相关原文.输出格式为:<结论>:结论\n\n<符合性分析>:符合性分析\n\n<原文>:原文"
    elif Hierarchy == "形成指定内容":
        messages = f"<相关文件>:{input_content}</相关文件>\n\n在上述相关文件中,是否有直接展示了‘{requirements}’对应的要求的内容? 即该部分实现了上述要求所要求的内容,而不是提出了对应的要求,而是实现了实际的内容.请给出结论,符合性分析,和带文件名称的相关原文.输出格式为:<结论>:结论\n\n<符合性分析>:符合性分析\n\n<原文>:原文"
    elif Hierarchy == "形成程序文件":
        messages = f"<相关文件>:{input_content}</相关文件>\n\n在上述相关文件中,是否有直接展示了‘{requirements}’对应的二级程序文件? 该二级文件应该非常详细和直接规定了上述要求对应的管理程序的各种细节,而不仅仅是一个要求.请给出结论,符合性分析,和带文件名称的相关原文.输出格式为:<结论>:结论\n\n<符合性分析>:符合性分析\n\n<文件>:文件名称及核心内容"
    elif Hierarchy == "形成其他文件":
        messages = f"<相关文件>:{input_content}</相关文件>\n\n<参考样式>{rag_content}\n\n</参考样式>在上述原文中,是否有直接展示了‘{requirements}’对应的要求的所形成的文件? 我期望的内容如<参考样式>,即该部分实现了上述要求阶段所输出的结果,而不是提出了对应的要求,也不是管理流程,而是实现了实际产出的内容.请给出结论,符合性分析,和带文件名称的相关原文.输出格式为:<结论>:结论\n\n<符合性分析>:符合性分析\n\n<文件>:文件名称及核心内容"
    elif Hierarchy == "形成记录":
        messages = f"<相关文件>:{input_content}</相关文件>在上述原文中,是否有直接展示了如何将‘{requirements}’对应的要求的形成记录? 我需要形成记录的内容, 形成记录的方式以及最后记录文件的名称,如有的话再提供已经形成的记录的表格或者表头.请给出结论,符合性分析,和带文件名称的相关原文.输出格式为:<结论>:结论\n\n<符合性分析>:符合性分析\n\n<记录>:记录表格名称及核心内容"
    return messages

/root/miniconda3/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [5]:
def enhanced_data_rag(QMS_file_name,enhaced_model_name,test_number=0,Skip_repetitive=True):
    ## enhaced_model_name should be "chat_internlm2_5","chat_qwen2_5"
    if test_number != 0:
        print("开始RAG!")
    elif test_number == 0:
        print("开始正式RAG!")
        print("检测文件是否已RAG!")
        output_file_path = f"../RAGed_data/output_{QMS_file_name}_{enhaced_model_name}.csv"
        if os.path.exists(output_file_path):
            print("该文件已RAG!")
            if Skip_repetitive:
                print("跳过该文件!")
                return None
            else:
                print("重复RAG并覆盖该文件!")
        else:
            print("当前文件未RAG!开始审核RAG!")
            print(f"当前QMS为{QMS_file_name},数据增强模型为{enhaced_model_name}!")

    
    df_RAG = pd.read_csv(f"../RAG_data/RAG_content_file_{QMS_file_name}.csv",encoding="utf-8")
    text_list = df_RAG['content']
    Index = torch.load(f'../RAG_data/RAG_Index_file_{QMS_file_name}.pth')
    df_requirements = pd.read_csv(f"../enhanced_data/enhanced_data_{enhaced_model_name}.csv",encoding="utf-8")
    
    enhanced_data_rag_list = []

    for i in tqdm(range(len(df_requirements))):
        Hierarchy = df_requirements['Hierarchy'][i]
        rag_content = df_requirements['enhanced_content'][i]
        requirements = df_requirements['requirements'][i]
    
        ## expert
        retrieve_dict = retriever(model_embedding, model_reranker, rag_content, Index, text_list, num_top = 10,num_top_rerank = 3)
        input_content = ""
        for key, value in retrieve_dict.items():
            input_content = input_content + '\n\n' + retrieve_dict[key]['content']
        enhanced_data_rag_list.append(input_content.strip())
        if test_number > 0:
            if i ==test_number-1:
                df_requirements = df_requirements.head(len(enhanced_data_rag_list))
                df_requirements['enhanced_data_rag_list'] = enhanced_data_rag_list
                df_requirements.to_csv(f"../RAGed_data/test_enhanced_rag_{QMS_file_name}_{enhaced_model_name}.csv")
                print("测试结果保存成功!")
                break

    if test_number == 0:
        df_requirements['enhanced_data_rag_list'] = enhanced_data_rag_list
        df_requirements.to_csv(output_file_path)
        print("审核结果保存成功!")

In [6]:
QMS_list = ["ZD","BJ", "MB"]
enhanced_model_list = ["chat_internlm2_5","chat_qwen2_5","chat_ERNIE","chat_Qwen"]
for QMS in QMS_list:
    for enhance_model in enhanced_model_list:
        enhanced_data_rag(QMS,enhance_model)

开始正式RAG!
检测文件是否已RAG!
该文件已RAG!
跳过该文件!
开始正式RAG!
检测文件是否已RAG!
当前文件未RAG!开始审核RAG!
当前QMS为ZD,数据增强模型为chat_qwen2_5!


/tmp/ipykernel_2868/1638514433.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  Index = torch.load(f'../RAG_data/RAG_Index_file_{QMS_file_name}.pth')
100%|██████████| 29

审核结果保存成功!
开始正式RAG!
检测文件是否已RAG!
当前文件未RAG!开始审核RAG!
当前QMS为ZD,数据增强模型为chat_ERNIE!


100%|██████████| 294/294 [00:23<00:00, 12.59it/s]
/tmp/ipykernel_2868/1638514433.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  Index = torch.load(f'../RAG_data/RAG_In

审核结果保存成功!
开始正式RAG!
检测文件是否已RAG!
当前文件未RAG!开始审核RAG!
当前QMS为ZD,数据增强模型为chat_Qwen!


...

审核结果保存成功!
开始正式RAG!
检测文件是否已RAG!
当前文件未RAG!开始审核RAG!
当前QMS为MB,数据增强模型为chat_ERNIE!


100%|██████████| 294/294 [00:22<00:00, 12.96it/s]
/tmp/ipykernel_2868/1638514433.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  Index = torch.load(f'../RAG_data/RAG_In

审核结果保存成功!
开始正式RAG!
检测文件是否已RAG!
当前文件未RAG!开始审核RAG!
当前QMS为MB,数据增强模型为chat_Qwen!


100%|██████████| 294/294 [00:23<00:00, 12.50it/s]

审核结果保存成功!
